In [39]:
from random import choice, randint, randrange, random
#import numpy as np

In [40]:
# (mutation, selection and crossover)

## INPUTS
# instrument <- "clarinet", "guitar", "brass", "piano"
# tempo
# duration per notes [seconds] <- float
# number of notes <- int
# pitch mean, 60
# pitch margin, +-5
# population
# number of mutations per gnome
# probability for mutation to occur

# NOT YET IMPLEMENTED
#num_generations = 50
#num_parents_mating = 4


In [89]:
instrument = "piano"
duration = 0.5 # [s]
number_of_notes = 12
total_duration = duration*number_of_notes
# tempo = 120
pitch_mean = 60
pitch_margin = 5
population_size = 5
mutations_per_gnome = 3 # number of mutations per gnome
mutation_probability = 0.1 # probability for mutation to occur

class GeneticAlgorithm:
    def __init__(self, 
                 instrument, 
                 duration, 
                 number_of_notes, 
                 pitch_mean, 
                 pitch_margin, 
                 population_size, 
                 mutations_per_gnome, 
                 mutation_probability
                ):
        
        self.instrument = instrument
        self.duration = duration
        self.number_of_notes = number_of_notes
        self.pitch_mean = pitch_mean
        self.pitch_margin = pitch_margin
        self.population_size = population_size
        self.mutations_per_gnome = mutations_per_gnome
        self.mutation_probability = mutation_probability

        self.total_duration = self.duration*self.number_of_notes
        self.pitch_min = self.pitch_mean-self.pitch_margin
        self.pitch_max = self.pitch_mean+self.pitch_margin
        

    def generate_genome(self):
        return [randint(self.pitch_min, self.pitch_max) for _ in range(self.number_of_notes)]
    
    def generate_population(self):
        return [self.generate_genome() for _ in range(self.population_size)]

    # crossover two parent genomes to create two children
    def crossover(self, p1, p2): # single point crossover
        if len(p1) == len(p2):
            p1_copy = p1.copy()
            p2_copy = p2.copy()
            
            length = len(p1_copy)
            ci = randint(1, length-1) # crossover index
            return (p1_copy[0:ci] + p2_copy[ci:], p2_copy[0:ci]+p1_copy[ci:])
        else:
            print("parent genomes must be of the same size")

    def mutation(self, genome): # it changes the gnome directly
        genome_copy = genome.copy()
        for _ in range(self.mutations_per_gnome):
            if random() > self.mutation_probability:
                index = randint(0,len(genome)-1)
                mutation_multiplier = choice([-1,1])
                mutation_amount = 1
                genome_copy[index] = genome_copy[index] + mutation_multiplier*mutation_amount
           
        return genome_copy
    
genetic_algo = GeneticAlgorithm(instrument, 
                                duration,
                                number_of_notes,
                                pitch_mean,
                                pitch_margin,
                                population_size,
                                mutations_per_gnome,
                                mutation_probability)

population = genetic_algo.generate_population()
p1, p2 = genetic_algo.crossover(population[0], population[1])
genome_mutated = genetic_algo.mutation(p1)


In [1]:
# genetic algorithm search of the one max optimization problem
from numpy.random import randint
from numpy.random import rand

# objective function
def onemax(x):
	return -sum(x)

# tournament selection
def selection(pop, scores, k=3):
	# first random selection
	selection_ix = randint(len(pop))
	for ix in randint(0, len(pop), k-1):
		# check if better (e.g. perform a tournament)
		if scores[ix] < scores[selection_ix]:
			selection_ix = ix
	return pop[selection_ix]

# crossover two parents to create two children
def crossover(p1, p2, r_cross):
	# children are copies of parents by default
	c1, c2 = p1.copy(), p2.copy()
	# check for recombination
	if rand() < r_cross:
		# select crossover point that is not on the end of the string
		pt = randint(1, len(p1)-2)
		# perform crossover
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]

# mutation operator
def mutation(bitstring, r_mut):
	for i in range(len(bitstring)):
		# check for a mutation
		if rand() < r_mut:
			# flip the bit
			bitstring[i] = 1 - bitstring[i]

# genetic algorithm
def genetic_algorithm(objective, n_bits, n_iter, n_pop, r_cross, r_mut):
	# initial population of random bitstring
	pop = [randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
	# keep track of best solution
	best, best_eval = 0, objective(pop[0])
	# enumerate generations
	for gen in range(n_iter):
		# evaluate all candidates in the population
		scores = [objective(c) for c in pop]
		# check for new best solution
		for i in range(n_pop):
			if scores[i] < best_eval:
				best, best_eval = pop[i], scores[i]
				print(">%d, new best f(%s) = %.3f" % (gen,  pop[i], scores[i]))
		# select parents
		selected = [selection(pop, scores) for _ in range(n_pop)]
		# create the next generation
		children = list()
		for i in range(0, n_pop, 2):
			# get selected parents in pairs
			p1, p2 = selected[i], selected[i+1]
			# crossover and mutation
			for c in crossover(p1, p2, r_cross):
				# mutation
				mutation(c, r_mut)
				# store for next generation
				children.append(c)
		# replace population
		pop = children
	return [best, best_eval]

# define the total iterations
n_iter = 100
# bits
n_bits = 20
# define the population size
n_pop = 100
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / float(n_bits)
# perform the genetic algorithm search
best, score = genetic_algorithm(onemax, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
print('f(%s) = %f' % (best, score))

>0, new best f([1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1]) = -12.000
>0, new best f([0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1]) = -13.000
>0, new best f([1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1]) = -14.000
>0, new best f([1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1]) = -15.000
>0, new best f([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1]) = -16.000
>3, new best f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1]) = -17.000
>4, new best f([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]) = -18.000
>6, new best f([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -19.000
>8, new best f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -20.000
Done!
f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -20.000000
